In [78]:
import pandas as pd
import numpy as np

data_identification = pd.read_csv("dm2023-isa5810-lab2-homework/data_identification.csv", skiprows=1, header=None, names=["tweet_id", "identification"])
emotion = pd.read_csv("dm2023-isa5810-lab2-homework/emotion.csv", skiprows=1, header=None, names=["tweet_id", "emotion"])

print(data_identification)
emotion

         tweet_id identification
0        0x28cc61           test
1        0x29e452          train
2        0x2b3819          train
3        0x2db41f           test
4        0x2a2acc          train
...           ...            ...
1867530  0x227e25          train
1867531  0x293813          train
1867532  0x1e1a7e          train
1867533  0x2156a5          train
1867534  0x2bb9d2          train

[1867535 rows x 2 columns]


,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation
...,...,...
1455558,0x38dba0,joy
1455559,0x300ea2,joy
1455560,0x360b99,fear
1455561,0x22eecf,joy


In [79]:
import json
tweets_DM=[]
for line in open('dm2023-isa5810-lab2-homework/tweets_DM.json', 'r'):
    tweets_DM.append(json.loads(line))

In [80]:
# 提取資料
extract_data = []
for tweet in tweets_DM:
    extract_data.append({
        'hashtags': tweet['_source']['tweet']['hashtags'],
        'tweet_id': tweet['_source']['tweet']['tweet_id'],
        'text': tweet['_source']['tweet']['text'],
        '_score': tweet['_score']
    })

# 將資料轉換成 DataFrame
extract_df = pd.DataFrame(extract_data)

extract_df = pd.merge(extract_df, data_identification, on='tweet_id')
# extract_df = pd.merge(extract_df, emotion, on='tweet_id')
# 顯示 DataFrame
extract_df

,hashtags,tweet_id,text,_score,identification
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",391,train
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",433,train
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",232,test
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,376,train
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",989,test
...,...,...,...,...,...
1867530,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...,827,test
1867531,[],0x29d0cb,I swear all this hard work gone pay off one da...,368,test
1867532,[],0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...,498,test
1867533,[],0x24faed,"Ah, corporate life, where you can date <LH> us...",840,train


In [81]:
train_data_identification_df = extract_df.loc[extract_df["identification"] == 'train']
train_data_identification_df  = train_data_identification_df .drop(['identification'],axis=1)
train_data_identification_df = pd.merge(train_data_identification_df, emotion, on='tweet_id')
# train_data_identification_list = list(train_data_identification_df['tweet_id'])
test_data_identification_df = extract_df.loc[extract_df["identification"] == 'test']
test_data_identification_df  = test_data_identification_df .drop(['identification'],axis=1)
# test_data_identification_list = list(test_data_identification_df['tweet_id'])

In [82]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer

# build analyzers (bag-of-words)
BOW_500 = CountVectorizer(max_features=31000, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW_500.fit(train_data_identification_df['text'])

c:\Users\sw_ya\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(max_features=31000,
                tokenizer=<function word_tokenize at 0x000001EF22BA81F0>)

In [83]:
# for a classificaiton problem, you need to provide both training & testing data
X_train = BOW_500.transform(train_data_identification_df['text'])
y_train = train_data_identification_df['emotion']

X_test = BOW_500.transform(test_data_identification_df['text'])

In [84]:
from sklearn.preprocessing import LabelEncoder
import keras
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print( label_encoder.classes_)


def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

Y_train = label_encode(label_encoder, y_train)

['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']


In [85]:
from sklearn.neural_network import MLPRegressor
NN=MLPRegressor(activation='logistic',solver='adam',alpha=0.00007,max_iter=15,hidden_layer_sizes=(32,32,32),verbose=True,early_stopping=False)
NN=NN.fit(X_train,Y_train)

Iteration 1, loss = 0.04082417
Iteration 2, loss = 0.03890460
Iteration 3, loss = 0.03875424
Iteration 4, loss = 0.03849323
Iteration 5, loss = 0.03826735
Iteration 6, loss = 0.03809216
Iteration 7, loss = 0.03799105
Iteration 8, loss = 0.03793369
Iteration 9, loss = 0.03792113
Iteration 10, loss = 0.03794250
Iteration 11, loss = 0.03798590
Iteration 12, loss = 0.03804211
Iteration 13, loss = 0.03811251
Iteration 14, loss = 0.03820995
Iteration 15, loss = 0.03829197


c:\Users\sw_ya\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


In [86]:
mlp_pre=NN.predict(X_train)
mlp_pre=label_decode(label_encoder,mlp_pre)

In [ ]:
mlp_pre_test=NN.predict(X_test)
mlp_pre_test=label_decode(label_encoder,mlp_pre_test)

In [ ]:
output5= pd.DataFrame({'id':test_data_identification_df .tweet_id , 'emotion': mlp_pre_test})
output5.to_csv('mlp_predict.csv', index=False)